# Imports

 Cover fundamentals of tensors using TensorFlow.
 
- Introduction to tensors
- Getting information from tensors
- Manipulating tensors
- Tensors and NumPy
- Using @tf.function (a way to speed up python)
- Using GPUs with TensorFlow (or TPUs)
- Exercises to try for yourself


## Introduction to Tensors

In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub

In [3]:
print(tf.__version__)

2.14.0


In [6]:
# Creating Tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [7]:
# Check number of dimensions of tensor, ndim
scalar.ndim

0

In [8]:
# Create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [9]:
vector.ndim

1

In [12]:
# create a matrix
matrix = tf.constant([[10,7],[7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [14]:
matrix.ndim

2

In [22]:
# create another matrix 
another_matrix = tf.constant([[[10.,7.],
                              [3.,2.]],
                              [[8.,9.],
                              [6.,2.]]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(2, 2, 2), dtype=float16, numpy=
array([[[10.,  7.],
        [ 3.,  2.]],

       [[ 8.,  9.],
        [ 6.,  2.]]], dtype=float16)>

In [23]:
another_matrix.ndim

3

In [24]:
tensor = tf.constant([[[1,2,3],
                       [4,5,6]],
                      [[7,8,9],
                       [10,11,12]],
                      [[13,14,15],
                       [16,17,18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [25]:
tensor.ndim

3

What we've created so far:
- scalar : single number
- vector: a number with direction
- matrix: 2-dimensional array of numbers
- tensor: n-dimensional array of numbers

### Creating tensors with tf.Variable

In [29]:
# create the same tensor but changeable
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [33]:
 # let's try and change one of the variables
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

- use constant if you dont want the variables to be changed
- use Variable when you want to change variables within the tensor
- Generally: When in doubt use constant and change later if you need

## Creating random Tensors

 Random tensors are tensors of some arbitrary size with random numbers

In [35]:
# Create random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42)
random_1 = random_1.normal(shape=(3,2))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.75658023, -0.06854693],
       [ 0.07595028, -1.2573844 ],
       [-0.23193759, -1.8107857 ]], dtype=float32)>

In [36]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.75658023, -0.06854693],
       [ 0.07595028, -1.2573844 ],
       [-0.23193759, -1.8107857 ]], dtype=float32)>

In [38]:
random_2 == random_1

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>

### Shuffle the order of elements in a tensor

In [39]:
# valuable when you want to shuffle data so the inherent order doesn't effect learning
not_shuffled = tf.constant([[10,7],
                            [3,4],
                            [2,5]])
not_shuffled.ndim

2

In [40]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [80]:
# need a global and operation level seeds
tf.random.set_seed(42)
shuffled = tf.random.shuffle(not_shuffled, seed=42)
shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

### Other ways to create tensors

#### Creating tensors from NumPy arrays

In [81]:
tf.ones([10,7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [82]:
tf.zeros(shape=(3,4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

Tensors can be run on a GPU, that is the main difference to NumPy array

In [85]:
# turn numpy arrays into tensors
numpy_A = np.arange(1,25, dtype=np.int32)

# X = tf.constant(some_matrix) -- capital for matrix
# y = tf.constant(vector) -- lower case for vector

In [89]:
A = tf.constant(numpy_A, shape=(2,3,4)) # product of new shape has to have the same length of original matrix/vector
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

### Getting Info from Tensors

- Shape -- tensor.shape
- Rank --  tensor.ndim
- Axis and dimension -- tensor[0], tensor[:,1]...
- Size -- tf.size(tensor)

In [91]:
# Create a rank 4 tensor
rank_4_tensor = tf.zeros([2, 3, 4, 5])

In [100]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor) # size = 2*3*4*5

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [105]:
# Get various attributes of tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
tensor_size = tf.size(rank_4_tensor)
print("Total number of elements in our tensor:", tensor_size.numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: 120


### Indexing on Tensors

Just like python lists

In [107]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [113]:
# get the first dimension from each index, except for the final one
rank_4_tensor[:1,:1, :1, :0]

<tf.Tensor: shape=(1, 1, 1, 0), dtype=float32, numpy=array([], shape=(1, 1, 1, 0), dtype=float32)>

In [115]:
# Create a rank 2 tensor
rank_2_tensor = tf.constant([[10, 7],
                             [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [117]:
# Get last item of each row
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [118]:
# Adding an extra dimension
rank_3_tensor = rank_2_tensor[..., tf.newaxis] # ... every axis before the last one
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [119]:
# Alternative
tf.expand_dims(rank_2_tensor, axis=-1)

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

### Manipulating Tensors (tensor operations)

#### Basic Operations

In [123]:
# addition operator
tensor = tf.constant([[10, 7],
                      [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [124]:
# Multiplication
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [125]:
# Subtraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [126]:
# tensorflow function (library)
tf.multiply(tensor, 10)
# Use the function!

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [127]:
tf.add(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [128]:
tf.subtract(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

### Matrix Multiplication

In machine learning, this concept is very common

#### Dot Product

In [129]:
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [130]:
tf.matmul(tensor, tensor) # if yo use tensor * tensor it is element wise

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [132]:
tensor_2 = tf.constant([[1, 2, 5],
                        [7, 2, 1],
                        [3, 3, 3]])
tensor_2

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 2, 5],
       [7, 2, 1],
       [3, 3, 3]], dtype=int32)>

In [133]:
tensor_3 = tf.constant([[3, 5],
                        [6, 7],
                        [1, 8]])

In [134]:
tf.matmul(tensor_2, tensor_3)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [136]:
# python operator @
tensor_2 @ tensor_3

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [137]:
# inverse of matrix
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])

In [141]:
Y = tf.transpose(Y)

In [145]:
tf.matmul(X, Y)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [146]:
# Can also use reshape(X, shape=(2,3))